# Room Schedule

Schedule teachers to rooms at their preferred times. In this system, **Events** (*sections*) are scheduled based on the limited availability of **Slots** (*rooms*). The time scale is at a week-to-week granularity, so special exceptions to the schedule must be handled outside of the system.

A user specifies how many sections they would like to teach and lists their preferred times through a separate form or interface. The data is then collected and onboarded as a Pandas DataFrame. Additional transformations are applied as necessary to massage the data into the right format so that the problem can be properly formulated.

## Setup

Import the required packages into the namespace.

In [ ]:
from conference_scheduler import scheduler
from conference_scheduler.resources import Slot, Event

from datetime import date, time, datetime, timedelta
from dateutil import parser, rrule
import numpy as np
import pandas as pd
import yaml

from collections import OrderedDict
from typing import NamedTuple
import itertools

### Conference Scheduler extensions

Add support for `Slot.starts_at` time preferences.

1. Subclass `conference_scheduler.resources.Event` to carry the `datetime` (not `Slot`!) preference data.
2. Define a new minimizing objective function for PuLP.

In [ ]:
class Course(NamedTuple):
    name: str
    duration: int
    demand: int

COURSES = {
    'CS 61A': Course('CS 61A', 60, 4),
    'CS 61B': Course('CS 61B', 60, 6),
    'CS 70' : Course('CS 70',  90, 6),
    'EE 16A': Course('EE 16A', 90, 6),
}

class Section(Event):
    """A section that needs to be scheduled."""

    # Preferred, Backup, If need be
    penalties, baseline_penalty = (0, 1.5), 3

    def __init__(self, name, course, tags=None, unavailability=None, preferences=None):
        """
        Section constructor. Course is one of the defined keys in COURSES while preferences is a
        zip of each list of start times with their corresponding penalties.
        """
        self.course = COURSES[course]
        super().__init__(name, self.course.duration, self.course.demand, tags, unavailability)
        if preferences is None:
            preferences = []
        self._preferences = tuple(preferences)

    def penalty(self, slot):
        for starts, penalty in self._preferences:
            if slot.starts_at in starts:
                return penalty
        return self.baseline_penalty

In [ ]:
def section_preference(slots, events, X, **kwargs):
    """
    Returns a function that calculates the section preference, taking into account the
    total difference between demand for an event and the slot capacity it is scheduled in.
    """
    sections = events
    return sum(max(section.penalty(slot) + (section.demand - slot.capacity), 0) * X[row, col]
               for row, section in enumerate(sections)
               for col, slot in enumerate(slots))

## Input data

### Generate `Slot` objects

Define the room availability for each room for the timeslots in which they're normally available each week. For each room availability range, we can divide them up into both 1-hour or 1.5-hour long blocks.

In [ ]:
START       = datetime(2018, 2, 5, 8)
END         = START + timedelta(days=5)
VALID_HOURS = range(8, 12 + 8)
PERIODS     = {
    60: rrule.rrule(rrule.HOURLY, byhour=VALID_HOURS,
                    dtstart=START, until=END, cache=True),
    90: rrule.rrule(rrule.MINUTELY, interval=90, byhour=VALID_HOURS,
                    dtstart=START, until=END, cache=True),
}

def starts(time_range, periods=PERIODS.items(), start=START):
    """Yield the start and duration for each periodic subdivision of the time range."""
    start, end = (parser.parse(time, default=start) for time in time_range.split('/'))
    assert start < end
    for duration, rule in periods:
        for starts_at in rule.between(start, end, inc=True):
            if starts_at + timedelta(minutes=duration) <= end:
                yield starts_at, duration

Data is imported from a `rooms.yml` YAML data file in the following format.

```yaml
Soda 283F:
    capacity: 4
    availability:
        - Mon 8am / Mon 12pm
```

The `availability` time range can be in any `dateutil.parser`-supported format separated by forward slash.

In [ ]:
with open('rooms.yml') as f:
    rooms = yaml.load(f)

slots = [
    Slot(room_name, starts_at, duration, room['capacity'], session=None)
    for room_name, room in rooms.items()
    for availability in room['availability']
    for starts_at, duration in starts(availability)
]

### Generate `Section` objects

Data is collected from a Google Form with the following column definitions.

In [ ]:
EMAIL     = 'Email Address'
COURSE    = 'Course'
ROOMS     = 'Rooms Desired'
PREFERRED = 'Preferred'
BACKUP    = 'Backup'
BASELINE  = 'Other'
DAYS      = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday')

Load the data into the environment from the exported `preferences.csv` and compress the preferences so that duplicate columns are merged.

In [ ]:
preferences = pd.read_csv('preferences.csv', dtype=str)
preferences = pd.concat([
    preferences[[EMAIL, COURSE, ROOMS]].rename(columns={EMAIL: 'Email', ROOMS: 'Rooms'}),
    pd.DataFrame(
        data=OrderedDict(
            (f'{pref}_{day}',
             (preferences
              .loc[:, preferences.columns.str.startswith(f'{pref} [{day}]')]
              .stack()
              .reset_index(1, drop=True)
             ))
            for pref in (PREFERRED, BACKUP, BASELINE) for day in DAYS
        )
    )
], axis=1).fillna('')

Define a function which can take a single row in the submitted by a user and expand it into as many `Section` objects as necessary. Preferences are expressed as `datetime` objects while the `unavailability` is given as a Python list of incompatible `Slot` and `Section` objects.

In [ ]:
def bounds(columns, column):
    """
    Return a two-element array containing the indices bounding the
    column from columns, including an offset for the index.
    """
    return np.where(columns.str.startswith(column))[0][[0, -1]]

def parse_datetimes(timestrs, days=DAYS, start=START):
    """Return the datetimes from the given days and time strings."""
    return [parser.parse(f'{day} {time}', default=start)
            for day, times in zip(days, timestrs)
            for time in times.split(', ')]

def make_sections(columns, slots, extract=[PREFERRED, BACKUP, BASELINE]):
    """Return a function which will yield Sections."""
    slices = {
        column: slice(*bounds(columns, column)) for column in extract
    }

    def kwargs_from_row(row, penalties=Section.penalties):
        """Return the keyword arguments to be passed into the Section constructor."""
        kwargs = dict()
        kwargs['name'] = row.email
        kwargs['course'] = row.course
        kwargs['tags'] = [row.email]
        preferences = [parse_datetimes(row[slices[column]]) for column in extract]
        kwargs['unavailability'] = [
            slot for slot in slots if slot.starts_at not in set().union(*preferences)
        ]
        kwargs['preferences'] = zip(preferences, penalties)
        return kwargs

    def sections_from_row(row):
        """Yield a Section for each desired room."""
        kwargs = kwargs_from_row(row)
        for _ in range(int(row.rooms)):
            yield Section(**kwargs)

    return sections_from_row

sections_from_row = make_sections(preferences.columns, slots)

In [ ]:
sections = list(
    itertools.chain.from_iterable(
        sections_from_row(row)
        for row in preferences.rename(columns=str.lower).itertuples(index=False)
    )
)

## Solve the schedule

In [ ]:
schedule = scheduler.schedule(sections, slots, objective_function=section_preference)

Format the schedule into a DataFrame and display the first few results.

In [ ]:
room_schedule = pd.DataFrame.from_records([
    (
        item.event.name,
        item.event.course.name,
        item.slot.venue,
        item.slot.capacity,
        item.slot.starts_at
    )
    for item in schedule
], columns=['Email Address', 'Course', 'Room', 'Capacity', 'Time'])

room_schedule.head()

### Export schedule

Export the results as a CSV to `room_schedule.csv` for importing to Google Sheets.

In [ ]:
room_schedule.to_csv('room_schedule.csv', index=False, date_format='%a %I:%M %p')